# Iodine Spectrometer Data Collection

In [ ]:
import time
from time import perf_counter, sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydaqmx_helper.adc import ADC

### Variable Naming

In [ ]:
sample_rate = 5000  # samples per second
bin_size = 0.05  # seconds per bin
countdown = 5  # seconds to wait before starting acquisition

In [ ]:
stop = False
def _enter_to_stop():
    global stop
    input("Press Enter to stop...\n")
    stop = True

In [ ]:
for s in range(countdown, 0, -1): # counting down so i know when to start the motor
    print(s)
    sleep(1)
print("Starting acquisition...")


In [ ]:
loop_stop = 0

myADC = ADC()
myADC.addChannels([0], minRange = -10, maxRange = 10)
data = myADC.sampleVoltages(sample_rate * bin_si

while loop_stop == 0:



In [ ]:
myADC = ADC()
myADC.addChannels([0], minRange = -10, maxRange = 10)


In [ ]:
import time
import numpy as np
import pandas as pd
from pydaqmx_helper.adc import ADC

# --- User settings ---
CHANNEL = 0
V_RANGE = (-10, 10)
SAMPLE_RATE = 5000
BIN_SIZE = 0.05  # seconds
DURATION = float(input("Scan duration (s): ") or "120")
LABEL = input("Label: ") or "run"

# --- Press Enter to stop ---
stop = False
def _enter_to_stop():
    global stop
    input("Press Enter to stop...\n")
    stop = True

import threading
threading.Thread(target=_enter_to_stop, daemon=True).start()

# --- Countdown ---
for s in range(5, 0, -1):
    print(s)
    time.sleep(1)
print("Starting acquisition...")

adc = ADC()
adc.addChannels([CHANNEL], minRange=V_RANGE[0], maxRange=V_RANGE[1])

samples_per_bin = int(SAMPLE_RATE * BIN_SIZE)
n_bins = int(DURATION / BIN_SIZE)
results = []

start = time.perf_counter()
while len(results) < n_bins and not stop:
    data = adc.sampleVoltages(samples_per_bin, SAMPLE_RATE)
    if np.ndim(data) > 1:
        data = data[:, 0]
    t = time.perf_counter() - start
    results.append([t, np.mean(data), np.std(data)])

df = pd.DataFrame(results, columns=["time_s", "mean_V", "std_V"])
fname = f"{LABEL}_{int(time.time())}.csv"
df.to_csv(fname, index=False)
print(f"Saved: {fname}")

In [ ]:
# ============================
# IODINE SPECTROMETER ACQUIRE
# single-channel (AI0), no OOP
# ============================

import time
from time import perf_counter, sleep
from pathlib import Path
import threading
import numpy as np
import pandas as pd
from pydaqmx_helper.adc import ADC   # your device helper

# ---------- USER SETTINGS ----------
SAVE_FOLDER            = Path("./data")   # where CSVs go
ADC_CHANNEL            = 0                # only channel 0
ADC_INPUT_RANGE_VOLTS  = (-10.0, 10.0)    # ±10 V
FAST_SAMPLE_RATE_HZ    = 5000             # raw sampling (safe for ~10 kS/s device)
BIN_DURATION_SECONDS   = 0.05             # average window (0.05 s -> ~20 Hz)
READ_BLOCK_SECONDS     = 0.20             # size of streaming read chunks
COUNTDOWN_SECONDS      = 5                # time to start motor before we begin
DROP_LAST_PARTIAL_BIN  = True             # keep uniform spacing
SAVE_FOLDER.mkdir(parents=True, exist_ok=True)

# ---------- PRESS-ENTER-TO-STOP ----------
STOP = False
def _enter_to_stop():
    global STOP
    try:
        input("Press Enter to STOP acquisition at any time...\n")
    except Exception:
        pass
    STOP = True

# ---------- SMALL UTILS ----------
def _save_csv(df: pd.DataFrame, path: Path, meta: dict):
    """Write a header + CSV so each file is self-describing."""
    header = [
        f"# Saved: {time.strftime('%Y-%m-%d %H:%M:%S')}",
        f"# Channel: {meta.get('channel')}",
        f"# Fast sample rate: {meta.get('fs_fast')} Hz",
        f"# Bin duration: {meta.get('bin_s')} s",
        f"# Input range: {meta.get('range_V')} V",
        f"# Planned duration: {meta.get('planned_s')} s",
        f"# Acquisition wall time: {meta.get('wall_s'):.3f} s",
        f"# Label: {meta.get('label')}",
        f"# Start wavelength (nm): {meta.get('start_nm')}",
        f"# End wavelength (nm): {meta.get('end_nm')}",
        f"# Notes: {meta.get('notes')}",
    ]
    path.write_text("\n".join(header) + "\n" + df.to_csv(index=False), encoding="utf-8")
    print(f"Saved: {path.resolve()}")

# ---------- CORE ACQUISITION ----------
def acquire_one_run(planned_duration_s,
                    label="run",
                    start_nm="",
                    end_nm="",
                    notes="",
                    fs_fast=FAST_SAMPLE_RATE_HZ,
                    bin_s=BIN_DURATION_SECONDS,
                    read_block_s=READ_BLOCK_SECONDS,
                    channel=ADC_CHANNEL,
                    v_range=ADC_INPUT_RANGE_VOLTS,
                    countdown_s=COUNTDOWN_SECONDS):
    """
    Stream raw samples from ADC channel 0, average into time-bins,
    return DataFrame with columns: time_s, ch0_mean_V, ch0_std_V
    """
    global STOP
    STOP = False

    # Countdown so you can start the motor
    print("\nGet ready to start the motor.")
    for s in range(countdown_s, 0, -1):
        print(f"  Starting in {s}…")
        sleep(1)
    print("Acquisition started.")

    # Start background "Enter to stop" listener
    threading.Thread(target=_enter_to_stop, daemon=True).start()

    # Pre-compute sizes
    samples_per_bin = int(round(fs_fast * bin_s))
    if samples_per_bin < 2:
        raise ValueError("BIN_DURATION_SECONDS too small for chosen FAST_SAMPLE_RATE_HZ.")
    block_samples = max(1, int(round(fs_fast * read_block_s)))

    # Configure ADC
    adc = ADC()
    adc.addChannels([channel], minRange=v_range[0], maxRange=v_range[1])

    # Rolling buffers for raw samples and their times
    raw = np.empty(0, dtype=float)
    t_raw = np.empty(0, dtype=float)
    sample_index = 0

    # Binned results
    t_mid_list, mean_list, std_list = [], [], []

    t0 = perf_counter()

    try:
        while True:
            elapsed = perf_counter() - t0
            if elapsed >= planned_duration_s or STOP:
                break

            # Read a chunk from the ADC (blocking call)
            block = np.asarray(adc.sampleVoltages(block_samples, fs_fast), dtype=float)
            if block.ndim > 1:  # safeguard: single-channel should be (N,), but handle (N,1)
                block = block[:, 0]

            # Append samples and their timestamps
            idx = np.arange(sample_index, sample_index + block.size, dtype=float)
            raw  = np.concatenate([raw, block])
            t_raw = np.concatenate([t_raw, idx / fs_fast])
            sample_index += block.size

            # Form as many full bins as possible
            full_bins = raw.size // samples_per_bin
            if full_bins > 0:
                upto = full_bins * samples_per_bin
                vals  = raw[:upto].reshape(full_bins, samples_per_bin)
                times = t_raw[:upto].reshape(full_bins, samples_per_bin)

                t_mid_list.extend(times.mean(axis=1).tolist())
                mean_list.extend(vals.mean(axis=1).tolist())
                std_list.extend(vals.std(axis=1, ddof=1).tolist())

                # remove processed samples
                raw  = raw[upto:]
                t_raw = t_raw[upto:]

        # Optionally fold the last partial bin (kept off by default)
        if (not DROP_LAST_PARTIAL_BIN) and raw.size >= 2:
            t_mid_list.append(t_raw.mean())
            mean_list.append(raw.mean())
            std_list.append(raw.std(ddof=1) if raw.size > 1 else 0.0)

    finally:
        wall = perf_counter() - t0
        try:
            del adc
        except Exception:
            pass

    # Pack into a DataFrame
    df = pd.DataFrame({
        "time_s": np.asarray(t_mid_list, dtype=float),
        f"ch{channel}_mean_V": np.asarray(mean_list, dtype=float),
        f"ch{channel}_std_V":  np.asarray(std_list,  dtype=float)
    })

    # Save file
    stamp = time.strftime("%Y%m%d-%H%M%S")
    fname = f"{stamp}_{label}_start{start_nm}nm_end{end_nm}nm_bins{bin_s:.3f}s_fs{fs_fast}Hz.csv"
    out_path = SAVE_FOLDER / fname
    meta = dict(channel=channel, fs_fast=fs_fast, bin_s=bin_s, range_V=v_range,
                planned_s=planned_duration_s, wall_s=wall,
                label=label, start_nm=start_nm, end_nm=end_nm, notes=notes)
    _save_csv(df, out_path, meta)

    print(f"Acquisition complete. Planned={planned_duration_s:.1f}s, wall={wall:.1f}s, points={len(df)}")
    return df, out_path

# ---------- INTERACTIVE MULTI-RUN LAUNCHER ----------
# Fill these once, then it will loop and save N runs.
label      = input("Label (e.g. reference_lamp / iodine_sample): ").strip() or "run"
start_nm   = input("Start wavelength on dial (nm): ").strip()
end_nm     = input("End wavelength on dial (nm): ").strip()
dur_s      = float(input("Planned scan duration (s) [e.g. 120]: ").strip() or "120")
notes      = input("Notes (motor voltage, slit widths, etc.): ").strip()
n_runs     = int(input("How many runs to acquire?: ").strip() or "1")

for i in range(1, n_runs+1):
    print(f"\n=== Run {i}/{n_runs}: {label} ===")
    df, path = acquire_one_run(planned_duration_s=dur_s,
                               label=label, start_nm=start_nm, end_nm=end_nm, notes=notes)

print("\nAll requested runs finished.")
